<a href="https://colab.research.google.com/github/Kabin119245/AI-ML/blob/main/fine_tune_whisper_parliament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ In

In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install kagglehub
!pip install pandas
!pip install gradio
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hbh43vz1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hbh43vz1
  Resolved https://github.com/huggingface/transformers to commit 816f4424964c1a1631e303b663fc3d68f731e923
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9991549 sha256=a3ea0d51f7aeb63cee89d2cda47c27f3b017c897eab99d5dff5b8e1870769a81
  Stored in directory: /tmp/pip-ephem-wheel-cache-ud10tiqo/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [3]:
!pip install pyxlsb

In [4]:
import os
import kagglehub
import pandas as pd
from datasets import DatasetDict, load_dataset, Dataset
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
import librosa


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import os
import kagglehub
import shutil

# Download dataset (this will download to the default cache location)
path = kagglehub.dataset_download("ishworsubedii/nepali-speech-to-text-dataset")

# Print the default path where the dataset has been downloaded
print(f"Dataset downloaded to cache: {path}")



100%|██████████| 1.01G/1.01G [00:59<00:00, 18.1MB/s]

Extracting files...


Dataset downloaded to cache: /root/.cache/kagglehub/datasets/ishworsubedii/nepali-speech-to-text-dataset/versions/1


In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define destination path in Google Drive
drive_path = '/content/drive/My Drive/nepali-speech-to-text-dataset'

# # Ensure the directory exists
os.makedirs(drive_path, exist_ok=True)

# # Move the dataset from the cache to Google Drive
shutil.move(path, drive_path)

print(f"Dataset moved to Google Drive at: {drive_path}")

Mounted at /content/drive
Dataset moved to Google Drive at: /content/drive/My Drive/nepali-speech-to-text-dataset


In [8]:
# Define paths in Google Drive
audio_dir = "/content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks"
transcript_file = "/content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/transcripts/audio transcript.xlsb"

# Load the transcription file using pandas and pyxlsb
transcriptions = pd.read_excel(transcript_file, engine='pyxlsb')

# Check the first few rows of the transcriptions to ensure it's loaded correctly
print("Transcriptions DataFrame:")
print(transcriptions.head())



Transcriptions DataFrame:
                                               Video  \
0  Nepali Speech To Text Dataset\audio_chunks\207...   
1  Nepali Speech To Text Dataset\audio_chunks\207...   
2  Nepali Speech To Text Dataset\audio_chunks\207...   
3  Nepali Speech To Text Dataset\audio_chunks\207...   
4  Nepali Speech To Text Dataset\audio_chunks\207...   

                                         Transcripts  
0  महेश कुमार महरा । समय ६ मिनेट । यसपछि माननीय द...  
1  सम्माननीय अध्यक्ष महोदय, आज म यो सदनमा अलि केह...  
2  मैले देखिरहेको छैन । एउटा म घटना क्रम यहाँ राख...  
3  म यो सम्मानित सदनबाट सम्माननीय अध्यक्षमार्फत्‌...  
4  चैं भर्ना गरिएको छ । र त्यहाँ न सङ्गठन बोल्छ न...  


In [9]:
# Function to clean the 'Video' column to extract only the file name
def clean_video_column(video_column):
    # Replace backslashes with forward slashes, then extract the filename (last part after '/')
    return video_column.apply(lambda x: os.path.basename(x.replace('\\', '/')))

# Load the transcription file
transcript_file = "/content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/transcripts/audio transcript.xlsb"
transcriptions = pd.read_excel(transcript_file, engine='pyxlsb')

# Clean the 'Video' column to get only the correct audio file names
transcriptions['cleaned_video'] = clean_video_column(transcriptions['Video'])

# Check the cleaned filenames
print("Cleaned Video column from the transcriptions:")
print(transcriptions['cleaned_video'].head())


Cleaned Video column from the transcriptions:
0    2079-11-27_1.wav 
1    2079-11-27_2.wav 
2    2079-11-27_3.wav 
3    2079-11-27_4.wav 
4    2079-11-27_5.wav 
Name: cleaned_video, dtype: object


In [10]:
# Function to normalize filenames (strip spaces, lowercase)
def normalize_filename(filename):
    return filename.strip().lower()

# Normalize the filenames in transcriptions and audio_files
transcriptions['normalized_video'] = transcriptions['cleaned_video'].apply(normalize_filename)

# List all audio files in the directory
audio_files = os.listdir(audio_dir)

# Normalize the actual audio filenames
normalized_audio_files = [normalize_filename(f) for f in audio_files]

# Print a few normalized filenames from both sides
print("Normalized video filenames from transcriptions:")
print(transcriptions['normalized_video'].head(20))

print("\nNormalized audio files from directory:")
print(normalized_audio_files[:20])  # Print the first 20 audio files

# Compare the two sets
missing_files = set(transcriptions['normalized_video']) - set(normalized_audio_files)
print(f"\nMissing files (after normalization): {missing_files}")


Normalized video filenames from transcriptions:
0      2079-11-27_1.wav
1      2079-11-27_2.wav
2      2079-11-27_3.wav
3      2079-11-27_4.wav
4      2079-11-27_5.wav
5      2079-11-27_6.wav
6      2079-11-27_7.wav
7      2079-11-27_8.wav
8      2079-11-27_9.wav
9     2079-11-27_10.wav
10    2079-11-27_11.wav
11    2079-11-27_12.wav
12    2079-11-27_13.wav
13    2079-11-27_15.wav
14    2079-11-27_16.wav
15    2079-11-27_17.wav
16    2079-11-27_18.wav
17    2079-11-27_19.wav
18    2079-11-27_20.wav
19    2079-11-27_21.wav
Name: normalized_video, dtype: object

Normalized audio files from directory:
['2079-11-21_26.wav', '2079-11-27_230.wav', '2079-11-27_79.wav', '2079-11-27_5.wav', '2079-11-27_166.wav', '2079-11-27_54.wav', '2079-11-27_55.wav', '2079-11-27_111.wav', '2079-11-27_97.wav', '2079-11-27_168.wav', '2079-11-27_80.wav', '2079-11-27_206.wav', '2079-11-21_46.wav', '2079-11-21_38.wav', '2079-11-27_4.wav', '2079-11-27_116.wav', '2079-11-27_75.wav', '2079-11-27_157.wav', '2079-11-2

In [11]:
# Modified function to store only audio paths and transcriptions
def load_audio_transcription_pairs(audio_dir, transcriptions, audio_files):
    data = {"path": [], "sentence": []}

    # Normalize filenames (convert to lowercase and strip spaces)
    available_files = set(normalize_filename(f) for f in audio_files)

    # Loop through each row in the transcriptions DataFrame using the cleaned video column
    for idx, row in transcriptions.iterrows():
        audio_file = normalize_filename(row['cleaned_video'])  # Normalize the cleaned video filename

        # Check if the file exists in the directory
        if audio_file not in available_files:
            print(f"Skipping missing file: {audio_file}")
            continue

        transcription = row['Transcripts']  # Transcription from the 'Transcripts' column

        # Find the actual file path by matching case-insensitively
        actual_file_name = next((f for f in audio_files if normalize_filename(f) == audio_file), None)
        if actual_file_name is None:
            print(f"Error: Could not find actual file for {audio_file}")
            continue

        # Build the complete audio file path
        audio_path = os.path.join(audio_dir, actual_file_name)

        # Append the audio path and transcription to the dataset
        data["path"].append(audio_path)
        data["sentence"].append(transcription)

    return data

# Load the dataset, skipping missing files
dataset_dict = load_audio_transcription_pairs(audio_dir, transcriptions, audio_files)

# Check the first few entries to confirm
print(f"Sample audio path: {dataset_dict['path'][5]}")
print(f"Sample transcription: {dataset_dict['sentence'][5]}")


Sample audio path: /content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks/2079-11-27_6.wav
Sample transcription: 

 दरबन्दी चैं प्रदेश प्रमुख कार्यालयमा थियो । सहायक हबल्दार हुनुहुन्छ । तर उहाँप्रति किन राज्य बेखबर भयो? अहिले चन्दा उठाएर उपचार गरेको अवस्था छ । कसैले दान गरेको भरमा दुई हजार, तीन हजार यस्तै दाताहरूको भरमा एउटा सुरक्षाकर्मी, एउटा राष्ट्रको सेवक कर्मचारीले त्यो अस्पतालको शय्यामा चैं जिन्दगी जिउनु विवश भएका छन्‌ । अब राज्यले त्यो उपचारको व्यवस्था सम्पूर्ण राज्यले लिनुपर्छ । 


In [12]:
# Check if any of the paths or sentences are invalid (e.g., None, NaN)
def clean_data(dataset_dict):
    cleaned_data = {"path": [], "sentence": []}
    for i in range(len(dataset_dict["path"])):
        audio_path = dataset_dict["path"][i]
        transcription = dataset_dict["sentence"][i]

        # Ensure both path and sentence are not None or NaN
        if isinstance(audio_path, str) and isinstance(transcription, str):
            cleaned_data["path"].append(audio_path)
            cleaned_data["sentence"].append(transcription)
        else:
            print(f"Skipping invalid entry at index {i}: {audio_path}, {transcription}")

    return cleaned_data

# Clean the dataset dictionary
cleaned_dataset_dict = clean_data(dataset_dict)

# Check cleaned data
print(f"Sample cleaned audio path: {cleaned_dataset_dict['path'][0]}")
print(f"Sample cleaned transcription: {cleaned_dataset_dict['sentence'][0]}")


Skipping invalid entry at index 98: /content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks/2079-11-27_101.wav, nan
Sample cleaned audio path: /content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks/2079-11-27_1.wav
Sample cleaned transcription: महेश कुमार महरा । समय ६ मिनेट । यसपछि माननीय दिल कुमारी
रावल ।


In [13]:
from datasets import Dataset, Audio

# Create a list of dictionaries with only the valid file paths and sentences
formatted_data = []
for i in range(len(cleaned_dataset_dict["path"])):
    formatted_data.append({
        "path": cleaned_dataset_dict["path"][i],  # Audio file path
        "sentence": cleaned_dataset_dict["sentence"][i]  # The transcription
    })

# Convert the list of dictionaries into a Hugging Face Dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.rename_column("path", "audio")
# Use the Audio feature to handle audio files directly from disk
dataset = dataset.cast_column("audio", Audio())

# Now split the dataset into training and test sets (80% train, 20% test)
split_datasets = dataset.train_test_split(test_size=0.2)

# Create a DatasetDict for train and test datasets
common_voice = DatasetDict({
    "train": split_datasets["train"],
    "test": split_datasets["test"]
})

# Check the structure of the dataset
print(common_voice)




DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 266
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 67
    })
})


In [14]:
# Access the first example in the training dataset
first_example = common_voice['train'][0]

# Print the audio data and its corresponding sentence (transcription)
print("Audio:", first_example['audio'])
print("Sentence:", first_example['sentence'])

Audio: {'path': '/content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks/2079-11-27_252.wav', 'array': array([-0.12969971, -0.17633057, -0.14248657, ..., -0.01641846,
       -0.01690674, -0.0133667 ]), 'sampling_rate': 44100}
Sentence:  ऐन, २०५९ लाई संशोधन गर्नका निम्ति प्रस्तुत भएको विधेयक जसमा सार्वजनिक संस्थाको दायरालाई फराकिलो बनाउने, राष्ट्र सेवकको  परिभाषालाई पनि अझ विस्तरित गर्ने र भ्रष्टाचारजन्य कसुर र सजायसम्बन्धी व्यवस्थालाई पुनर्व्यवस्थित गर्नेलगायतका प्रावधानहरू राखेर आएको 


In [15]:
print(common_voice["train"][0])

{'audio': {'path': '/content/drive/MyDrive/nepali-speech-to-text-dataset/1/Nepali Speech To Text Dataset/audio_chunks/2079-11-27_252.wav', 'array': array([-0.12969971, -0.17633057, -0.14248657, ..., -0.01641846,
       -0.01690674, -0.0133667 ]), 'sampling_rate': 44100}, 'sentence': ' ऐन, २०५९ लाई संशोधन गर्नका निम्ति प्रस्तुत भएको विधेयक जसमा सार्वजनिक संस्थाको दायरालाई फराकिलो बनाउने, राष्ट्र सेवकको  परिभाषालाई पनि अझ विस्तरित गर्ने र भ्रष्टाचारजन्य कसुर र सजायसम्बन्धी व्यवस्थालाई पुनर्व्यवस्थित गर्नेलगायतका प्रावधानहरू राखेर आएको '}


In [16]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [17]:
model_name_or_path="openai/whisper-large-v3"
language = "Nepali"
language_abbr = "ne-NP"
task = "transcribe"

In [18]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [19]:
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [20]:
def prepare_dataset(batch):

    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"],truncation=True, max_length=448).input_ids
    return batch

In [21]:
# Apply the preprocessing function to the dataset
common_voice = common_voice.map(prepare_dataset, remove_columns=["audio", "sentence"])

# Check the structure of the dataset after preprocessing
print(common_voice)


Map:   0%|          | 0/266 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 266
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 67
    })
})


In [22]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [23]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [24]:
import evaluate

metric = evaluate.load("wer")

In [25]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [26]:
from transformers import WhisperForConditionalGeneration, BitsAndBytesConfig
model_name_or_path = "openai/whisper-large-v3"

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True))

# model.hf_device_map - this should be {" ": 0}

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [27]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [28]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [29]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 15,728,640 || all params: 1,559,219,200 || trainable%: 1.0088


In [33]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-NEPALI-LARGE-v3-parliamentdata-3",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=50,
    num_train_epochs=3,
    eval_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],
    push_to_hub=True# same reason as above
)

In [31]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

<ipython-input-31-fb2b4ea414ad>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [32]:
processor.save_pretrained(training_args.output_dir)

[]

In [34]:
from accelerate import Accelerator

In [ ]:
accelerator = Accelerator()
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarni

Epoch,Training Loss,Validation Loss
1,0.213900,0.371528
2,0.193200,0.384837
3,0.136900,0.392423
4,0.088800,0.414228
5,0.056900,0.442030


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=170, training_loss=0.13337679154732648, metrics={'train_runtime': 1756.0736, 'train_samples_per_second': 0.757, 'train_steps_per_second': 0.097, 'total_flos': 4.5668703338496e+18, 'train_loss': 0.13337679154732648, 'epoch': 5.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from huggingface_hub import HfApi
import os

# Set the path to the folder you want to upload (in Colab's filesystem)
folder_path = '/content/whisper-NEPALI-LARGE-v3-parliamentdata-2'

# Set the repository name on Hugging Face
repo_name = "girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-2"

# Create a Hugging Face repository (dataset or model)
api = HfApi()
api.create_repo(repo_name, private=False)  # Set private=True if you want the repo to be private

# Upload the folder to Hugging Face
api.upload_folder(
    folder_path=folder_path,
    repo_id=repo_name,
    repo_type="model"  # Use "dataset" for datasets or "model" for models
)


adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

events.out.tfevents.1729324494.4e1b96b48ac1.350.0:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-2/commit/fed802c17db09ebda05c83fb7b37c7eb8c42199c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='fed802c17db09ebda05c83fb7b37c7eb8c42199c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-2', endpoint='https://huggingface.co', repo_type='model', repo_id='girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-2'), pr_revision=None, pr_num=None)

In [35]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-3"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

In [36]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")

  0%|          | 0/9 [00:00<?, ?it/s]<ipython-input-36-25fc43e49d9b>:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `En

wer=50.75803517283202


In [37]:
!pip install gradio

In [38]:
import torch
import gradio as gr
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig


peft_model_id = "girikabin/whisper-NEPALI-LARGE-v3-parliamentdata-3"
language = "Nepali"
task = "transcribe"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
pipe = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text


iface = gr.Interface(
    fn=transcribe,  # Assuming 'transcribe' is your function that processes the audio input
    inputs=gr.Audio(type="filepath"),  # No need for source="microphone", it's implied
    outputs="text",
    title="PEFT LoRA + INT8 Whisper Large V2 Nepali By Kabin Giri",
    description="Realtime demo for Nepali speech recognition using `PEFT-LoRA+INT8` fine-tuned Whisper Large V2 model.",
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8698e974d67a566552.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
